In [1]:
! pip install statsmodels

You should consider upgrading via the 'c:\users\saidj\onedrive\documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\scripts\python.exe -m pip install --upgrade pip' command.


In [2]:
import os
import sys
root_dir = os.path.abspath(os.path.join(os.path.dirname('pruebillas.ipynb'), '..'))
os.chdir(root_dir)
sys.path.insert(0, os.path.join(root_dir, 'src'))

In [3]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover, plot_heatmaps, resample_apply, barssince
import pandas as pd
import talib as ta
import numpy as np

from backbone.utils.general_purpose import diff_pips

In [4]:
symbols_path = './backbone/data/backtest/symbols/USDCAD.csv'
df = pd.read_csv(symbols_path)
df

,Date,Open,High,Low,Close,Volume,direction,Group,consecutive_candles
0,2021-01-01 00:00:00,1.27316,1.27316,1.27316,1.27316,0.000000e+00,-1,1,1
1,2021-01-01 00:01:00,1.27316,1.27316,1.27316,1.27316,0.000000e+00,-1,1,2
2,2021-01-01 00:02:00,1.27316,1.27316,1.27316,1.27316,0.000000e+00,-1,1,3
3,2021-01-01 00:03:00,1.27316,1.27316,1.27316,1.27316,0.000000e+00,-1,1,4
4,2021-01-01 00:04:00,1.27316,1.27316,1.27316,1.27316,0.000000e+00,-1,1,5
...,...,...,...,...,...,...,...,...,...
1578235,2024-01-01 23:55:00,1.32475,1.32475,1.32473,1.32473,2.400000e+06,-1,553947,1
1578236,2024-01-01 23:56:00,1.32474,1.32479,1.32455,1.32455,3.360000e+07,-1,553947,2
1578237,2024-01-01 23:57:00,1.32456,1.32463,1.32447,1.32447,2.965000e+07,-1,553947,3
1578238,2024-01-01 23:58:00,1.32449,1.32450,1.32446,1.32446,9.750000e+06,-1,553947,4


In [21]:
train_start = '2023-01-01'
train_end = '2023-03-01'

test_start = '2023-04-01'
test_end = '2023-04-30'

train_data = df[(df.Date > train_start) & (df.Date < train_end)]
test_data = df[(df.Date > test_start) & (df.Date < test_end)]

In [6]:
def  optim_func(series):
    return series['Return [%]'] / (-1*series['Max. Drawdown [%]'])

In [7]:
class ConsecutiveCandles(Strategy):
    n = 7
    stop_loss_pips = 10 # pips
    take_profit_pips = 10 # pips
    pip_value = 0.0001
    position_size = 0.001

    def init(self):
        pass
    
    def next(self):
        close_prices = self.data.Close
        open_prices = self.data.Open

        actual_close_price = close_prices[-1]

        if barssince(close_prices < open_prices) == self.n:
            sl_price = round(actual_close_price - self.stop_loss_pips * self.pip_value, 6)
            tp_price = round(actual_close_price + self.take_profit_pips * self.pip_value, 6)
            
            self.buy(sl=sl_price, tp=tp_price, size=self.position_size)

        elif barssince(close_prices > open_prices) == self.n:
            sl_price = round(actual_close_price + self.stop_loss_pips * self.pip_value, 6)
            tp_price = round(actual_close_price - self.take_profit_pips * self.pip_value, 6)
            # print(f'short tp:{tp_price}, actual {actual_close_price}, sl {sl_price}')

            self.sell(sl=sl_price, tp=tp_price, size=self.position_size)


In [8]:
import numpy as np
import pandas as pd

def consecutive(close, open, n):
    # Determina la dirección de las velas: 1 para alcista, -1 para bajista
    direction = np.where(close > open, 1, -1)
    
    # Detecta cambios en la dirección
    changes = np.diff(direction, prepend=direction[0])
    group = np.cumsum(changes != 0)
    
    # Contar el número de velas consecutivas dentro de cada grupo
    consecutive_candles = np.zeros_like(group)
    unique_groups, counts = np.unique(group, return_counts=True)
    for grp, count in zip(unique_groups, counts):
        consecutive_candles[group == grp] = np.arange(1, count + 1)
    
    # Crear una serie con valores 0 por defecto
    result = np.zeros_like(direction)
    
    # Asignar 1 o -1 para secuencias que tienen exactamente n velas consecutivas
    result[consecutive_candles == n] = direction[consecutive_candles == n]
    
    return pd.Series(result)


In [30]:
from backtesting import Strategy

class ConsecutiveCandlesGrid(Strategy):
    n = 7  # Número de velas consecutivas
    percent_profit = 0.001  # Porcentaje de ganancia basada en el equity
    percent_loss = 0.05  # Porcentaje de pérdida basada en el equity
    pip_value = 0.0001
    grid_size = 15  # Tamaño de la cuadrícula (10 pips)
    position_size = 3000

    def init(self):
        self.consecutive = self.I(consecutive, self.data.Close, self.data.Open, n=self.n)
        self.rsi = self.I(ta.RSI, self.data.Close, 14)

    
    def next(self):
        close_prices = self.data.Close
        open_prices = self.data.Open
        actual_close_price = close_prices[-1]

        # Verificar si hay posiciones abiertas
        if self.position:
            profit_target = self.equity * self.percent_profit
            loss_target = self.equity * self.percent_loss

            total_profit = sum(trade.pl for trade in self.trades)
            last_trade = self.trades[-1]
            pips = diff_pips(last_trade.entry_price, actual_close_price, pip_value=self.pip_value, absolute=True)

            dinamic_grid_size = self.grid_size * (len(self.trades))

            # Cerrar todas las posiciones si se alcanza el profit o la pérdida
            if total_profit <= -loss_target or total_profit >= profit_target:
                self.position.close()

            # Añadir posición en caso de que el profit/loss entre dentro del rango
            elif self.position.is_long and pips > dinamic_grid_size:
                self.buy(size=self.position_size)
                    
            elif self.position.is_short and pips > dinamic_grid_size:
                self.sell(size=self.position_size)

        else:

            # Abrir posición en función de las velas consecutivas 
            today = self.data.Date[-1]
            volume = self.data.Volume
            if self.consecutive == -1 and volume < 300 and today.weekday != 5:
                self.buy(size=self.position_size)
                
            elif self.consecutive == 1 and volume < 300 and today.weekday != 5:
                self.sell(size=self.position_size)


In [10]:

bt_train = Backtest(
    train_data, 
    ConsecutiveCandlesGrid, 
    cash=15000, 
    commission=0.0002,
    margin=1/30
)


# stats = bt_train.run()
stats = bt_train.optimize(
    n=range(7, 10, 1),
    percent_profit=list(np.arange(0.0001, 0.001, 0.01)), # array([0.01, 0.02, 0.03, 0.04, 0.05])
    percent_loss=list(np.arange(0.01, 0.3, 0.01)), # array([0.01, 0.03, 0.05, 0.07, 0.09])
    grid_size=list(np.arange(10, 40, 10)),
    position_size=list(np.arange(1000, 5000, 1000)),
    # maximize='Max. Drawdown [%]',
    maximize=optim_func,
    max_tries=1000,
)




c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\backtesting.py:1054: FutureWarning: Index.is_numeric is deprecated. Use pandas.api.types.is_any_real_numeric_dtype instead
  (data.index.is_numeric() and
C:\Users\SaidJ\AppData\Local\Temp\ipykernel_22068\3556247800.py:1: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt_train = Backtest(
c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\backtesting.py:1488: UserWarning: Searching for best of 997 configurations.
  output = _optimize_grid()


  0%|          | 0/9 [00:00<?, ?it/s]

In [26]:
bt_train.plot()

stats

Start                               1051200.0
End                                 1136159.0
Duration                              84959.0
Exposure Time [%]                   88.600518
Equity Final [$]                 15163.311756
Equity Peak [$]                  15163.544686
Return [%]                           1.088745
Buy & Hold Return [%]                0.707713
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                   -0.432035
Avg. Drawdown [%]                   -0.008862
Max. Drawdown Duration                17374.0
Avg. Drawdown Duration             147.659696
# Trades                                143.0
Win Rate [%]                        74.125874
Best Trade [%]                       0.893747
Worst Trade [%]                     -0.538379
Avg. Trade [%]                    

In [12]:
stats._trades.groupby(by=['ExitTime']).agg({'PnL':['sum','count']})

PnL      
               sum count
ExitTime                
1052803   5.818348     2
1053321   1.587382     2
1054142   1.667154     2
1054579   2.745300     3
1054839   1.787524     1
...            ...   ...
1135201   1.508380     1
1135593   2.276528     2
1135615   1.677852     1
1135940   1.567692     1
1136159  -0.232930     1

[89 rows x 2 columns]

In [13]:
print(f'n: {stats._strategy.n}')
print(f'percent_profit: {stats._strategy.percent_profit}')
print(f'percent_loss: {stats._strategy.percent_loss}')
print(f'grid_size: {stats._strategy.grid_size}')
print(f'position_size: {stats._strategy.position_size}')

n: 7
percent_profit: 0.0001
percent_loss: 0.01
grid_size: 20
position_size: 1000


In [35]:
# test_data['Date'] = pd.to_datetime(test_data['Date'])
# test_data = test_data.set_index('Date')
bt_test = Backtest(
    train_data, 
    ConsecutiveCandlesGrid, 
    cash=15000, 
    commission=0.0002,
    margin=1/30
)
 
test_stats = bt_test.run(
    n=stats._strategy.n,
    percent_profit=stats._strategy.percent_profit,
    percent_loss=stats._strategy.percent_loss,
    grid_size=stats._strategy.grid_size,
    position_size=5000,
)

test_stats

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\backtesting.py:1054: FutureWarning: Index.is_numeric is deprecated. Use pandas.api.types.is_any_real_numeric_dtype instead
  (data.index.is_numeric() and
C:\Users\SaidJ\AppData\Local\Temp\ipykernel_22068\4167021533.py:3: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt_test = Backtest(


Start                               1051200.0
End                                 1136159.0
Duration                              84959.0
Exposure Time [%]                   35.486111
Equity Final [$]                   15103.5388
Equity Peak [$]                    15103.5388
Return [%]                           0.690259
Buy & Hold Return [%]                0.707713
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                   -0.283534
Avg. Drawdown [%]                   -0.026795
Max. Drawdown Duration                 7049.0
Avg. Drawdown Duration             466.039604
# Trades                                 47.0
Win Rate [%]                        82.978723
Best Trade [%]                       0.217014
Worst Trade [%]                     -0.064182
Avg. Trade [%]                    

In [36]:
trades = test_stats._trades.groupby(by=['ExitTime']).agg({'PnL':['sum','count'], 'Duration':'max'})
trades

PnL       Duration
               sum count      max
ExitTime                         
1052803   14.69590     1      195
1052915    1.93691     2       97
1053059    2.24445     1       60
1053109    3.69371     1       43
1053166    2.14515     1       21
1053952    1.69175     1      130
1054141    3.43565     2      161
1056961    2.70182     1       80
1062605    0.30494     1     2878
1064177    1.66050     1       78
1065598    1.65717     1      113
1068446    1.96307     1       64
1073256    1.82266     2     3451
1074392    2.55872     1      238
1075830    1.76079     1      252
1077160    2.90031     1      124
1082765    4.51193     1     2878
1083714    5.12336     2      925
1085669    1.56294     1       21
1087233    1.41108     1      135
1088857    1.91695     1      323
1092858    0.71843     1     2891
1093094    1.63718     2      225
1094539    1.81125     1      235
1102921    8.35989     1     2876
1103538    2.76739     2      604
1106062    1.51001     1      234
1113007    2.61538     1     2880
1113091    2.81495     1       77
1114639    2.06637     1      156
1116032    1.71589     1      139
1123102    1.70227     1     2895
1123158    1.75200     1       34
1124521    1.85491     1      220
1124701    2.75409     1      169
1126345    1.64595     1      378
1129057    1.69232     2      206
1133489    1.82891     2     3202
1135030    2.59185     1      422

In [37]:
bt_test.plot(filename='./ConsecutiveCandlesGrid.html', resample=False)

GridPlot(id='p9012', ...)

In [18]:
# plot_heatmaps(heatmap, agg='mean')zzz